In [ ]:
THIS_NOTEBOOK = RabbitMQ

In [ ]:
# import amqp

In [3]:
## Open Blocking Connection:
import pika


def open_connection():
    creds = pika.credentials.PlainCredentials(username='ub2',password='mq_money')
    connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost',port=5672,
                                                               virtual_host=None,credentials=creds))
    channel = connection.channel()
    return channel

In [24]:
ch = open_connection()
ch.basic_publish(exchange='celery',
                 routing_key='celery',
                 body="{'task':'tasks.update_master_repo','args':['/home/jail/home/serv/system_config/SERVER2/.git']}")
ch.close()

In [14]:
ch.basic_publish?

In [58]:
## Delete Exchanges/Queues:

ch = open_connection()
ch.exchange_delete(exchange='ngx')
ch.queue_delete(queue='ngx')
ch.queue_delete(queue='celery')
ch.close()

[TUTORIAL 1](https://www.rabbitmq.com/tutorials/tutorial-one-python.html)

In [ ]:
# SENDING:

channel = open_connection()

# create queue
channel.queue_declare(queue='hello')

# need to push message through exchange to get to queue
# this exchange is interesting:  queue name == routing name
channel.basic_publish(exchange='',
                      routing_key='hello',
                      body='Hello World!')
print " [x] Sent 'Hello World!'"
connection.close()

In [ ]:
# RECEIVING:

channel = open_connection()

# doesn't matter if declared multiple times
channel.queue_declare(queue='hello')
def callback(ch, method, properties, body):
    print " [x] Received %r" % (body,)

# queue must exist.. any check avail.?
channel.basic_consume(callback,
                      queue='hello',
                      no_ack=True)

# never ending consumption
print ' [*] Waiting for messages. To exit press CTRL+C'
channel.start_consuming()

[TUTORIAL 3 -- Publish/Subscribe](https://www.rabbitmq.com/tutorials/tutorial-three-python.html)

In [59]:
# GOAL: push messages from ngx to celery
"""
    1. create exchange ("ngx") to publish ngx messages
    2. bind exchange ("ngx") to queue ("celery")
    3. bind exchange ("celery",existing) to queue ("celery")
"""

# PUBLISH LOGS (from nginx)

channel = open_connection()
channel.exchange_declare(exchange='ngx',
                         exchange_type='direct',
                         durable=True)

channel.close()

# RECEIVE LOGS (@ celery)

queue_name='celery'

channel = open_connection()
result = channel.queue_declare(queue=queue_name,durable=True)
channel.queue_bind(
    queue=queue_name,
    exchange='ngx',
    routing_key='ngx_celery')
channel.close()

channel = open_connection()
result = channel.queue_declare(queue=queue_name,passive=True,durable=True)
channel.queue_bind(
    queue=queue_name,
    exchange='celery',
    routing_key='ngx_celery')
channel.close()

In [38]:
channel.queue_bind?

[TUTORIAL 5 -- Using Topic Exchanges](https://www.rabbitmq.com/tutorials/tutorial-five-python.html)
  
 ---- [Useful for Firehose Debugging/Tracing](https://www.rabbitmq.com/firehose.html)

In [ ]:
# PUBLISH

channel = open_connection()

channel.exchange_declare(exchange='topic_logs',
                         type='topic')

channel.basic_publish(exchange='topic_logs',
                      routing_key=routing_key,
                      body=message)
print " [x] Sent %r:%r" % (routing_key, message)
connection.close()

In [ ]:
# RECEIVE

channel = open_connection()
queue_name='document_processing.sync'
channel.queue_bind(exchange='celery',
                   queue=queue_name,
                   routing_key='document_processing.sync.#')